<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>


## Part 1:  (Web) Scrape Wikipedia Page on Toronto Neighbourhoods


#### Install Python Packages

In [1]:
#install Beatiful Soup (if not yet available)

!pip install beautifulsoup4

     |████████████████████████████████| 102kB 17.7MB/s ta 0:00:01


In [2]:
#install xml parser(if not yet available)

!pip install lxml

     |████████████████████████████████| 5.8MB 28.5MB/s eta 0:00:01


In [3]:
#install HTML5 library (if not yet available)

!pip install html5lib

#### Import Python Libraries

In [4]:
#import libraries


import pandas as pd
import requests
from bs4 import BeautifulSoup

import html

In [5]:
#get html and 'suoify' table

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content, 'lxml')#'html5lib')
#print(soup.prettify)

In [6]:
#access table

table = soup.find('table')
data = pd.read_html(str(table))

In [7]:
#check html to retrieved
data

[    Postcode           Borough          Neighbourhood
 0        M1A      Not assigned           Not assigned
 1        M2A      Not assigned           Not assigned
 2        M3A        North York              Parkwoods
 3        M4A        North York       Victoria Village
 4        M5A  Downtown Toronto           Harbourfront
 ..       ...               ...                    ...
 283      M8Z         Etobicoke              Mimico NW
 284      M8Z         Etobicoke     The Queensway West
 285      M8Z         Etobicoke  Royal York South West
 286      M8Z         Etobicoke         South of Bloor
 287      M9Z      Not assigned           Not assigned
 
 [288 rows x 3 columns]]

In [8]:
#place table contents into datafrme
df = data[0]

In [9]:
#verify data type

print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [10]:
#display dataframe

df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [11]:
#copy original dataframe in oder to process it
dfp = df

#dfp



#### Build the DataFrame

In [12]:
#set up the dataframe columns for appending

dfp_p = dfp['Postcode']
dfp_b = dfp['Borough']
dfp_n = dfp['Neighbourhood']

In [13]:
#convert series to dataframe

dfp_n = dfp_n.to_frame()
dfp_b = dfp_b.to_frame()
dfp_p = dfp_p.to_frame()

In [14]:
#combine 'slices' into a single dataframe

dfp_bn = dfp_b.merge(dfp_n,left_index=True, right_index=True)
dfp_pbn = dfp_p.merge(dfp_bn,left_index=True, right_index=True)

#### Handle Boroughs that are "Not assigned" 

In [15]:
#remove all Boroughs that are not assigned
dfp_pbn = dfp_pbn[dfp_pbn['Borough'] != "Not assigned"]

print(len(dfp_pbn))

211


#### Handle Neighbourhoods that are "Not assigned" 

In [16]:
#assign Borough value to Neigborhoods that are "Not Assigned"

#initilise variables used in the loop

i=0
nn=0

dfp_pbn_bna = dfp_pbn
rows = len(dfp_pbn_bna)

na = "Not assigned"

#loop through entire dataframe
while i < rows:
    b = dfp_pbn_bna['Borough'].values[i]
    n = dfp_pbn_bna['Neighbourhood'].values[i]
    n.strip()
    b.strip()
    if (n==na):
        nn ++ 1
        print("N: No borough!",i,b,n)
        dfp_pbn_bna['Neighbourhood'].values[i] = b
    i += 1    
        
print(nn,len(dfp_pbn_bna))

N: No borough! 6 Queen's Park Not assigned
0 211


In [17]:
print(dfp_pbn_bna.values[6])

['M7A' "Queen's Park" "Queen's Park"]


#### Handle Neiborhoods that belong to the same Postcode

In [18]:
#join Neighborhoods with the same Postcode

#setup the variables used in the loop
i=0
i2=0
np=0
dfp_pbn_bna_nna = dfp_pbn_bna
rows = len(dfp_pbn_bna_nna)


dfp_pbn_bna_nna.reset_index(drop=True, inplace=True)
#dfp_pbn_bna_nna.head()

dfp_pbn_bna_nna2 = dfp_pbn_bna_nna

#perform a nested loop to concatenate neighborhoods with similar postcodes
while i < rows:
    b = dfp_pbn_bna_nna['Borough'].values[i]
    n = dfp_pbn_bna_nna['Neighbourhood'].values[i]
    p = dfp_pbn_bna_nna['Postcode'].values[i]
    i2 = i + 1
    
    while i2 < rows:
        p2 = dfp_pbn_bna_nna['Postcode'].values[i2]
        if (p == p2): 
            n2 = dfp_pbn_bna_nna['Neighbourhood'].values[i2]
            dfp_pbn_bna_nna['Neighbourhood'].values[i] = n + ", "  +  n2
           
        i2 += 1
    i += 1    
        
print(np,len(dfp_pbn_bna_nna))

0 211


#### Finalise dataframe

In [19]:
dfp_pbn_bna_nna2

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge, Malvern"
9,M1B,Scarborough,Malvern


In [20]:
#remove duplicate rowa
dfp_pbn_bna_nna2.drop_duplicates(subset ="Postcode", keep = 'first', inplace = True) 

In [21]:
print(len(dfp_pbn_bna_nna2))

103


In [22]:
#reset dtaframe to start rows at 0
dfp_pbn_bna_nna2.reset_index(drop=True, inplace=True)

In [23]:
#rename coumn names
dfp_pbn_bna_nna2.rename(columns={"Postcode": "PostalCode", "Neighbourhood": "Neighborhood"},inplace=True)
dfp_pbn_bna_nna2

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


#### Final step (Part 1 of 3)

In [28]:
#display number of rows and columns
dfp_pbn_bna_nna2.shape

(103, 3)

## Part 2: Geospatial daea containing latitudes & longitudes

#### First get the csv to make it work

In [29]:
#download csv 

!wget -q -O 'TorHoods_Coord.csv' : http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [30]:
#put csv into dataframe
df_LatLong_csv = pd.read_csv('TorHoods_Coord.csv')
df_LatLong_csv.rename(columns={"Postal Code": "PostalCode"},inplace=True)

#df_LatLong_csv

## Make final dataframe

In [32]:
#get Boroughs
df_b = dfp_pbn_bna_nna2[['PostalCode','Borough']]
df_b


,PostalCode,Borough
0,M3A,North York
1,M4A,North York
2,M5A,Downtown Toronto
3,M6A,North York
4,M7A,Queen's Park
5,M9A,Etobicoke
6,M1B,Scarborough
7,M3B,North York
8,M4B,East York
9,M5B,Downtown Toronto


#### Boroughs

In [34]:
#add borough rows to original dataframe
df_m1 = df_LatLong_csv
df_m1 = df_m1.merge(df_b, how='inner',on ='PostalCode',left_index=True, right_index=True)
df_m1

,PostalCode,Latitude,Longitude,Borough
0,M1B,43.806686,-79.194353,North York
1,M1C,43.784535,-79.160497,North York
2,M1E,43.763573,-79.188711,Downtown Toronto
3,M1G,43.770992,-79.216917,North York
4,M1H,43.773136,-79.239476,Queen's Park
5,M1J,43.744734,-79.239476,Etobicoke
6,M1K,43.727929,-79.262029,Scarborough
7,M1L,43.711112,-79.284577,North York
8,M1M,43.716316,-79.239476,East York
9,M1N,43.692657,-79.264848,Downtown Toronto


#### Neighborhoods

In [35]:
#get Neighborhoods
df_n = dfp_pbn_bna_nna2[['PostalCode','Neighborhood']]
df_n


,PostalCode,Neighborhood
0,M3A,Parkwoods
1,M4A,Victoria Village
2,M5A,"Harbourfront, Regent Park"
3,M6A,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park
5,M9A,Islington Avenue
6,M1B,"Rouge, Malvern"
7,M3B,Don Mills North
8,M4B,"Woodbine Gardens, Parkview Hill"
9,M5B,"Ryerson, Garden District"


#### Combine into one

In [36]:
#add neighbourhood rows to intermediary dataframe
df_m2 = df_m1
df_m2 = df_m2.merge(df_n, how='inner',on ='PostalCode',left_index=True, right_index=True)
df_m2

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,North York,Parkwoods
1,M1C,43.784535,-79.160497,North York,Victoria Village
2,M1E,43.763573,-79.188711,Downtown Toronto,"Harbourfront, Regent Park"
3,M1G,43.770992,-79.216917,North York,"Lawrence Heights, Lawrence Manor"
4,M1H,43.773136,-79.239476,Queen's Park,Queen's Park
5,M1J,43.744734,-79.239476,Etobicoke,Islington Avenue
6,M1K,43.727929,-79.262029,Scarborough,"Rouge, Malvern"
7,M1L,43.711112,-79.284577,North York,Don Mills North
8,M1M,43.716316,-79.239476,East York,"Woodbine Gardens, Parkview Hill"
9,M1N,43.692657,-79.264848,Downtown Toronto,"Ryerson, Garden District"


#### Column order

In [37]:
#reorder columns of the dataframe 
df_m2[['PostalCode','Borough','Neighborhood','Latitude','Longitude']]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,North York,Parkwoods,43.806686,-79.194353
1,M1C,North York,Victoria Village,43.784535,-79.160497
2,M1E,Downtown Toronto,"Harbourfront, Regent Park",43.763573,-79.188711
3,M1G,North York,"Lawrence Heights, Lawrence Manor",43.770992,-79.216917
4,M1H,Queen's Park,Queen's Park,43.773136,-79.239476
5,M1J,Etobicoke,Islington Avenue,43.744734,-79.239476
6,M1K,Scarborough,"Rouge, Malvern",43.727929,-79.262029
7,M1L,North York,Don Mills North,43.711112,-79.284577
8,M1M,East York,"Woodbine Gardens, Parkview Hill",43.716316,-79.239476
9,M1N,Downtown Toronto,"Ryerson, Garden District",43.692657,-79.264848


#### Try using geocoder instead of csv

#### Make sure package is installed

In [38]:
!pip install geocoder

     |████████████████████████████████| 102kB 18.9MB/s ta 0:00:01
     |████████████████████████████████| 829kB 17.5MB/s eta 0:00:01
     |████████████████████████████████| 81kB 16.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future


#### Import library

In [39]:
# import geocoder
import geocoder 

#### Get one to test

In [40]:
bd = dfp_pbn_bna_nna2['Borough'].values[0]
print(bd)

North York


#### Use string format prescribed by the assement - some code on the Internet doesn't have the curly braces at the start

In [ ]:
bstr = '{}, ' + bd  + ', Ontario''.format(postal_code))'
print(bstr)

In [ ]:
# initialize your variable to None
lat_lng_coords = None

In [ ]:
# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google(bstr)
  lat_lng_coords = g.latlng

In [ ]:
latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

#### it's taking sometime to process just one so I'm sticking with the csv - end of Part 2